# Testing file 
### where we evaluate LAFTR and Zhang's models using the test set

## Preliminaries

In [1]:
from math import sqrt, isnan
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.optimizers import Adam, Adagrad
from tensorflow.data import Dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv

from util import metrics
from util.load_data import load_data
from util.evaluation import *

from zhang.models import FairLogisticRegression
from zhang.learning import train_loop as zhang_train

from madras_laftr.models import *
from madras_laftr.learning import train_loop as laftr_train

In [2]:
batch_size = 64
epochs = 1
learning_rate = 0.001
opt = Adam(learning_rate=learning_rate)

In [3]:
CLAS_COEFF = 1.
FAIR_COEFF = 1.
RECON_COEFF = 0.
hidden_layer_specs = {'clas':[8] , 'enc':[8] , 'dec':[8] , 'adv':[8]}

In [4]:
test_loop = 1

## Load data

In [5]:
x_train, y_train, a_train = load_data('adult', 'train')
raw_data = (x_train, y_train, a_train)

In [6]:
xdim = x_train.shape[1]
ydim = y_train.shape[1]
adim = a_train.shape[1]
zdim = 8

In [7]:
train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
train_data = train_data.batch(batch_size, drop_remainder=True)
train_data

<BatchDataset shapes: ((64, 113), (64, 1), (64, 1)), types: (tf.float64, tf.float64, tf.float64)>

In [8]:
x_valid, y_valid, a_valid = load_data('adult', 'valid')

valid_data = Dataset.from_tensor_slices((x_valid, y_valid, a_valid))
valid_data = valid_data.batch(batch_size, drop_remainder=True)

In [9]:
x_test, y_test, a_test = load_data('adult', 'test')

test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
test_data = test_data.batch(batch_size, drop_remainder=True)

## Result file

In [10]:
header = "model_name", "clas_acc", "dp", "deqodds", "deqopp", "trade_dp", "trade_deqodds", "trade_deqopp"
results = []

## Testing loop
#### Each model is evalueted 5 times
#### In the end of each iteration we save the result

### LAFTR for DP

In [11]:
for i in range(test_loop):
    model = DemParGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
    ret = laftr_train(model, raw_data, train_data, epochs, opt)

    Y_hat, A_hat = evaluation(model, valid_data)
    clas_acc, dp, deqodds, deqopp, confusion_matrix = compute_metrics(y_valid, Y_hat, a_valid, A_hat)

    fair_metrics = (dp, deqodds, deqopp)
    tradeoff = []
    for fair_metric in fair_metrics:
        tradeoff.append(compute_tradeoff(clas_acc, fair_metric))
    
    results.append(['LAFTR4DP', clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]])

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.13617490231990814 | 1.0443549156188965 | 0.6358301639556885 | 58318.01171875 | 0.7176724137931034 | 0.4946949602122016 | 7.46626326259947
> Class Acc | Adv Acc
> 0.7406914893617021 | 0.6717087765957447
> DP | DEqOdds | DEqOpp
> 0.9431509114801884 | 0.9386943206191063 | 0.9634782448410988
> Confusion Matrix 
TN: 4355.0 | FP: 163.0 
FN: 1397.0 | TP: 101.0
> Confusion Matrix for A = 0 
TN: 1659.0 | FP: 8.0 
FN: 211.0 | TP: 1.0
> Confusion Matrix for A = 1 
TN: 2696.0 | FP: 155.0 
FN: 1186.0 | TP: 100.0


### LAFTR for Eq Odds

In [12]:
for i in range(test_loop):
    model = EqOddsUnweightedGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
    ret = laftr_train(model, raw_data, train_data, epochs, opt)

    Y_hat, A_hat = evaluation(model, valid_data)
    clas_acc, dp, deqodds, deqopp, confusion_matrix = compute_metrics(y_valid, Y_hat, a_valid, A_hat)

    fair_metrics = (dp, deqodds, deqopp)
    tradeoff = []
    for fair_metric in fair_metrics:
        tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

    results.append(['LAFTR4EqOdds', clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]])

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | -0.049510080367326736 | 1.1509439945220947 | 1.2994742393493652 | 57436.18359375 | 0.7634698275862069 | 0.637309350132626 | 261.84379144562337
> Class Acc | Adv Acc
> 0.7844082446808511 | 0.7599734042553191
> DP | DEqOdds | DEqOpp
> 0.7798723876476288 | 0.8177158534526825 | 0.8795555830001831
> Confusion Matrix 
TN: 3851.0 | FP: 667.0 
FN: 630.0 | TP: 868.0
> Confusion Matrix for A = 0 
TN: 1551.0 | FP: 116.0 
FN: 133.0 | TP: 79.0
> Confusion Matrix for A = 1 
TN: 2300.0 | FP: 551.0 
FN: 497.0 | TP: 789.0


### LAFTR for Eq Opp

In [13]:
for i in range(test_loop):
    model = EqOddsUnweightedGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
    ret = laftr_train(model, raw_data, train_data, epochs, opt)

    Y_hat, A_hat = evaluation(model, valid_data)
    clas_acc, dp, deqodds, deqopp, confusion_matrix = compute_metrics(y_valid, Y_hat, a_valid, A_hat)

    fair_metrics = (dp, deqodds, deqopp)
    tradeoff = []
    for fair_metric in fair_metrics:
        tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

    results.append(['LAFTR4EqOpp', clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]])

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.2636153995990753 | 1.3502320051193237 | 0.5593857765197754 | 64874.7109375 | 0.7742042440318302 | 0.8128730106100795 | 45.182567970822284
> Class Acc | Adv Acc
> 0.8098404255319149 | 0.9271941489361702
> DP | DEqOdds | DEqOpp
> 0.8614690154790878 | 0.966437853872776 | 0.9896784126758575
> Confusion Matrix 
TN: 3941.0 | FP: 577.0 
FN: 567.0 | TP: 931.0
> Confusion Matrix for A = 0 
TN: 1503.0 | FP: 164.0 
FN: 84.0 | TP: 128.0
> Confusion Matrix for A = 1 
TN: 2438.0 | FP: 413.0 
FN: 483.0 | TP: 803.0


### Zhang for DP

In [14]:
fairdef = 'DemPar'

for i in range(test_loop):
    model = FairLogisticRegression(xdim, batch_size, fairdef)
    zhang_train(model, raw_data, train_data, epochs)

    Y_hat, A_hat = evaluation(model, valid_data)
    clas_acc, dp, deqodds, deqopp, confusion_matrix = compute_metrics(y_valid, Y_hat, a_valid, A_hat)

    fair_metrics = (dp, deqodds, deqopp)
    tradeoff = []
    for fair_metric in fair_metrics:
        tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

    results.append(['Zhang4DP', clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]])

> Epoch | Class Loss | Adv Loss | Class Acc | Adv Acc
> 1 | 0.4763365685939789 | 0.5228971242904663 | 0.7144811007957559 | 0.6152602785145889
> Class Acc | Adv Acc
> 0.7347074468085106 | 0.7292220744680851
> DP | DEqOdds | DEqOpp
> 0.4515211582183838 | 0.4934789538383484 | 0.7179714739322662
> Confusion Matrix 
TN: 3200.0 | FP: 1318.0 
FN: 278.0 | TP: 1220.0
> Confusion Matrix for A = 0 
TN: 1653.0 | FP: 14.0 
FN: 142.0 | TP: 70.0
> Confusion Matrix for A = 1 
TN: 1547.0 | FP: 1304.0 
FN: 136.0 | TP: 1150.0


### Zhang for Eq Odds

In [15]:
fairdef = 'EqOdds'

for i in range(test_loop):
    model = FairLogisticRegression(xdim, batch_size, fairdef)
    zhang_train(model, raw_data, train_data, epochs)

    Y_hat, A_hat = evaluation(model, valid_data)
    clas_acc, dp, deqodds, deqopp, confusion_matrix = compute_metrics(y_valid, Y_hat, a_valid, A_hat)

    fair_metrics = (dp, deqodds, deqopp)
    tradeoff = []
    for fair_metric in fair_metrics:
        tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

    results.append(['Zhang4EqOdds', clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]])

> Epoch | Class Loss | Adv Loss | Class Acc | Adv Acc
> 1 | 0.8880362510681152 | 0.7379366159439087 | 0.7536057692307692 | 0.3326840185676393
> Class Acc | Adv Acc
> 0.8070146276595744 | 0.3700132978723404
> DP | DEqOdds | DEqOpp
> 0.8402800112962723 | 0.8858708739280701 | 0.9221954941749573
> Confusion Matrix 
TN: 4132.0 | FP: 386.0 
FN: 775.0 | TP: 723.0
> Confusion Matrix for A = 0 
TN: 1601.0 | FP: 66.0 
FN: 138.0 | TP: 74.0
> Confusion Matrix for A = 1 
TN: 2531.0 | FP: 320.0 
FN: 637.0 | TP: 649.0


### Zhang for Eq Opp

In [16]:
fairdef = 'EqOpp'

for i in range(test_loop):
    model = FairLogisticRegression(xdim, batch_size, fairdef)
    zhang_train(model, raw_data, train_data, epochs)

    Y_hat, A_hat = evaluation(model, valid_data)
    clas_acc, dp, deqodds, deqopp, confusion_matrix = compute_metrics(y_valid, Y_hat, a_valid, A_hat)

    fair_metrics = (dp, deqodds, deqopp)
    tradeoff = []
    for fair_metric in fair_metrics:
        tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

    results.append(['Zhang4EqOpp', clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]])

> Epoch | Class Loss | Adv Loss | Class Acc | Adv Acc
> 1 | 0.4192975163459778 | 0.2229214310646057 | 0.7571286472148542 | 0.32949270557029176
> Class Acc | Adv Acc
> 0.8134973404255319 | 0.3407579787234043
> DP | DEqOdds | DEqOpp
> 0.8658372312784195 | 0.9245323911309242 | 0.9551923424005508
> Confusion Matrix 
TN: 4270.0 | FP: 248.0 
FN: 874.0 | TP: 624.0
> Confusion Matrix for A = 0 
TN: 1640.0 | FP: 27.0 
FN: 140.0 | TP: 72.0
> Confusion Matrix for A = 1 
TN: 2630.0 | FP: 221.0 
FN: 734.0 | TP: 552.0


## Saving into DF then CSV

In [17]:
result_df = pd.DataFrame(results, columns=header)
result_df

,model_name,clas_acc,dp,deqodds,deqopp,trade_dp,trade_deqodds,trade_deqopp
0,LAFTR4DP,0.740691,0.943151,0.938694,0.963478,0.829750,0.828020,0.837522
1,LAFTR4EqOdds,0.784408,0.779872,0.817716,0.879556,0.782134,0.800716,0.829262
2,LAFTR4EqOpp,0.809840,0.861469,0.966438,0.989678,0.834857,0.881236,0.890773
3,Zhang4DP,0.734707,0.451521,0.493479,0.717971,0.559312,0.590403,0.726243
4,Zhang4EqOdds,0.807015,0.840280,0.885871,0.922195,0.823311,0.844606,0.860769
5,Zhang4EqOpp,0.813497,0.865837,0.924532,0.955192,0.838852,0.865468,0.878669


In [18]:
result_df.to_csv('test_result_p1.csv')